# Overview

The data has been split into two groups:

* training set (train.csv)
* test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

## Data Dictionary

| Variable | Definition                                 | Key                                            |   |   |
|----------|--------------------------------------------|------------------------------------------------|---|---|
| survival | Survival                                   | 0 = No, 1 = Yes                                |   |   |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |   |   |
| sex      | Sex                                        |                                                |   |   |
| Age      | Age in years                               |                                                |   |   |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |   |   |
| parch    | # of parents / children aboard the Titanic |                                                |   |   |
| ticket   | Ticket number                              |                                                |   |   |
| fare     | Passenger fare                             |                                                |   |   |
| cabin    | Cabin number                               |                                                |   |   |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |   |   |
## Variable Notes

pclass: A proxy for socio-economic status (SES)
* 1st = Upper
* 2nd = Middle
* 3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...

Sibling = brother, sister, stepbrother, stepsister

Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father

Child = daughter, son, stepdaughter, stepson

Some children travelled only with a nanny, therefore parch=0 for them.

# Data cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Functions

In [2]:
def label_fare(fare):
    if(fare < 8):
        return "< 8"
    if(fare >= 8 and fare < 14.5):
        return "< 14.5"
    if(fare >= 14.5 and fare < 31):
        return "< 31"
    if(fare >= 31):
        return ">= 31"
    
    
def label_age(age):
    if(age < 3):
        return "baby"
    if(age >= 3 and age < 12):
        return "kid"
    if(age >= 12 and age < 18):  # Maybe we can ignore this label
        return "Teeneger"
    if(age >= 18 and age < 60):
        return "Adult"
    if(age >= 60):
        return "Old"
    
    
def label_class(pclass):
    if(pclass == 1):
        return "Upper"
    if(pclass == 2):
        return "Middle"
    if(pclass == 3):
        return "Lower"

In [3]:
from sklearn.preprocessing import OneHotEncoder

def categorial_feature(dataframe, column_name):
    cat_feature = dataframe[[column_name]].to_numpy()
    
    feature_encoder = OneHotEncoder()
    cat_feature = feature_encoder.fit_transform(cat_feature).toarray()

    dataframe.drop([column_name], axis=1, inplace=True)

    feature_encoder.categories_[0]
    cat_feature = pd.DataFrame(cat_feature, columns=feature_encoder.categories_[0])

    dataframe = pd.concat([dataframe,cat_feature] , axis=1)
    
    return dataframe


def pipeline(dataframe):
    
    dataframe["Age"] = dataframe["Age"].fillna(np.mean(dataframe["Age"]))
    
    if (dataframe["Fare"].isnull().values.any() == True):
        index_nans = dataframe["Fare"].index[dataframe["Fare"].apply(np.isnan)]
        class_nan = dataframe["Pclass"].loc[index_nans].to_numpy()[0]    
        mean_fare_class = dataframe.loc[dataframe["Pclass"] == class_nan]["Fare"].mean()
        dataframe["Fare"] = dataframe["Fare"].fillna(np.mean(dataframe["Fare"]))
        
        
    dataframe.drop(columns=["Cabin", "Embarked", "Name", "Ticket","SibSp", "Parch"],
                               inplace=True)
    
    dataframe["Fare"] = dataframe["Fare"].apply(label_fare)
    dataframe["Pclass"] = dataframe["Pclass"].apply(label_class)
    dataframe["Age"] = dataframe["Age"].apply(label_age)
    
    dataframe = categorial_feature(dataframe, "Sex")      
    dataframe = categorial_feature(dataframe, "Fare")
    dataframe = categorial_feature(dataframe, "Age")
    dataframe = categorial_feature(dataframe, "Pclass")    
    
    return dataframe

## Applying pipeline

In [4]:
train_df = pd.read_csv("train.csv")
test_sample = pd.read_csv("test.csv")

train_df.pop("PassengerId").to_numpy()
test_sample_ID = test_sample.pop("PassengerId").to_numpy()

train_df = pipeline(train_df)
test_sample = pipeline(test_sample)

In [5]:
# Counting how many NAN values there is on the dataframe
for column in train_df.columns:
    nan_counter = train_df[column].isna().sum()
    if (nan_counter != 0):
        print(f"{column} has {nan_counter} NAN values")

In [6]:
# Counting how many NAN values there is on the test dataframe
for column in test_sample.columns:
    nan_counter = test_sample[column].isna().sum()
    if (nan_counter != 0):
        print(f"{column} has {nan_counter} NAN values")

# Model

Looking for the best model

In [7]:
train_label =  train_df.pop("Survived").to_numpy()
train_feature = train_df.to_numpy()

test_sample = test_sample.to_numpy()

In [8]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import cross_val_predict

## KNeighborsClassifier

In [9]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {"n_neighbors": [8, 9, 10, 11, 12],
              "weights": ["uniform", "distance"]}

KNC = KNeighborsClassifier()

grid_search = GridSearchCV(KNC, param_grid, cv=10,
                        scoring='neg_mean_squared_error',
                        return_train_score=True)

grid_search.fit(train_feature, train_label)

KNC = grid_search.best_estimator_

In [10]:
n_scores = cross_val_score(KNC,
                           train_feature, train_label,
                           cv=10,
                           scoring="accuracy")

print(f"Mean scores = {n_scores.mean()*100:.3f}%")

Mean scores = 81.599%


# The following models do not perform better than the KNeighborsClassifier

## RandomForestClassifier

In [11]:
# from sklearn.ensemble import RandomForestClassifier

# param_grid = {"n_estimators": [900, 1000, 1100],
#               "criterion": ["entropy", "gini"]}


# RFR = RandomForestClassifier()

# grid_search = GridSearchCV(RFR, param_grid, cv=10,
#                         scoring='neg_mean_squared_error',
#                         return_train_score=True)

# grid_search.fit(train_feature, train_label)

# RFR = grid_search.best_estimator_

In [12]:
# n_scores = cross_val_score(RFR,
#                            train_feature, train_label,
#                            cv=3)

# print(f"Mean scores = {n_scores.mean()*100:.3f}%")

## SGDClassifier

In [13]:
# from sklearn.linear_model import SGDClassifier

# param_grid = {"penalty": ["l2", "l1", "elasticnet"],
#               "loss": ["perceptron", "log", "hinge"]}


# GDC = SGDClassifier()

# grid_search = GridSearchCV(GDC, param_grid, cv=10,
#                         scoring='neg_mean_squared_error',
#                         return_train_score=True)

# grid_search.fit(train_feature, train_label)

# GDC = grid_search.best_estimator_

In [14]:
# n_scores = cross_val_score(GDC,
#                            train_feature, train_label,
#                            cv=3)

# print(f"Mean scores = {n_scores.mean()*100:.3f}%")

# Predict

In [15]:
pred_labels = KNC.predict(test_sample)

s1 = pd.DataFrame(pd.Series(test_sample_ID), columns=["PassengerId"])
s2 = pd.DataFrame(pd.Series(pred_labels), columns=["Survived"])

final_results = s1.join(s2)

final_results.to_csv("Predicted.csv",  index=False)
final_results

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
